In [5]:
# Import stuff
import torch as t
import numpy as np
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import plotly.express as px
import einops
import plotly.graph_objects as go 
from functools import partial
import tqdm.auto as tqdm
import circuitsvis as cv
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, ActivationCache
from transformer_lens.components import Embed, Unembed, LayerNorm, MLP
from fancy_einsum import einsum
from jaxtyping import Float, Int, Bool
import re
import random


In [2]:

def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [3]:
## turn off AD to save memory, since we're focusing on model inference here 
t.set_grad_enabled(False)

device = 'cuda' if t.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained('gpt2-small', device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


Studying the head outputs on different distributions

Create the datasets 

In [87]:
proper_nouns = [
    " Goose", " Church",
    " Google", " Chair",
    " Bag", " Statue",
    " Lamp", " Flower"
]

nouns = [
    " goose", " church",
    " google", " chair",
    " bag", " statue",
    " lamp", " flower"
]

multi_names_religious = [
    " Mary", " Joseph",
    " Abraham", " Paul",
    " Isaac", " Noah",
    " Jacob", " Jesus"
]

multi_names_places = [
    " Paris", " London",
    " Madison", " Phoenix",
    " Devon", " Florence",
    " Austin", " Brooklyn"
]

test = proper_nouns
[model.to_single_token(test[i]) for i in range(len(test))]


[46317, 4564, 3012, 9369, 20127, 43330, 28607, 20025]

In [171]:



def get_dataset(N, names):
    prompts = []
    # List of answers, in the format (correct, incorrect)
    answers = []
    # List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
    answer_tokens = []
    for _ in range(N):
        S1, S2 = random.sample(names,2)
        answers.append((S1,S2))
        answers.append((S2,S1))
        prompt1 = f'When {S1} and {S2} went to the shops,{S2} gave the bag to'
        prompt2 = f'When {S1} and {S2} went to the shops,{S1} gave the bag to'
        prompts.append(prompt1)
        prompts.append(prompt2)

        answer_tokens.append((model.to_single_token(answers[-1][0]),model.to_single_token(answers[-1][1])))
        answer_tokens.append((model.to_single_token(answers[-1][1]),model.to_single_token(answers[-1][0])))
    assert len(set([len(model.to_str_tokens(prompt)) for prompt in prompts])) == 1
    answer_tokens = t.tensor(answer_tokens).to(device)
    return prompts, answers, answer_tokens



In [172]:
prompts_rel, ans_rel, ans_toks_rel = get_dataset(10,multi_names_religious)
prompts_pl, ans_pl, ans_toks_pl = get_dataset(10,multi_names_places)
prompts_n, ans_n, ans_toks_n = get_dataset(10,nouns)
prompts_pn, ans_pn, ans_toks_pn = get_dataset(10,proper_nouns)



In [173]:
from rich.table import Table, Column
from rich import print as rprint


In [174]:
control_prompts = ['When John and Mary went to the shops, John gave the bag to',
 'When John and Mary went to the shops, Mary gave the bag to',
 'When Tom and James went to the park, James gave the ball to',
 'When Tom and James went to the park, Tom gave the ball to',
 'When Dan and Sid went to the shops, Sid gave an apple to',
 'When Dan and Sid went to the shops, Dan gave an apple to',
 'After Martin and Amy went to the park, Amy gave a drink to',
 'After Martin and Amy went to the park, Martin gave a drink to']

control_answers = [(' Mary', ' John'),
 (' John', ' Mary'),
 (' Tom', ' James'),
 (' James', ' Tom'),
 (' Dan', ' Sid'),
 (' Sid', ' Dan'),
 (' Martin', ' Amy'),
 (' Amy', ' Martin')]

control_ans_prompts = t.tensor([[ 5335,  1757],
        [ 1757,  5335],
        [ 4186,  3700],
        [ 3700,  4186],
        [ 6035, 15686],
        [15686,  6035],
        [ 5780, 14235],
        [14235,  5780]])



In [175]:
print(model(prompts_n).shape)

def ave_logit_diff(prompts, answer_tokens, per_prompt = False):
    final_logits = model(prompts)[:,-1,:]
    answer_logits = final_logits.gather(dim = -1, index = answer_tokens)
    answer_logit_diff = answer_logits[:,0] - answer_logits[:,1]
    if per_prompt:
        return answer_logit_diff 
    else:
        return answer_logit_diff.mean()



torch.Size([20, 17, 50257])


In [176]:
def make_table(prompts, answers, answer_tokens, title):
    cols = [
        "Prompt", 
        Column("Correct", style="rgb(0,200,0) bold"), 
        Column("Incorrect", style="rgb(255,0,0) bold"), 
        Column("Logit Difference", style="bold")
    ]
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
    logit_diff_table = Table(*cols, title=title + f": Ave logit diff = {ave_logits.item():.3f}")
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
                                 
    ave_logit_diff(prompts, answer_tokens)
    for prompt, ans, logit_diff in zip(prompts, answers,logit_diffs):
        logit_diff_table.add_row(prompt, ans[0], ans[1], f"{logit_diff.item():.3f}")
    rprint(logit_diff_table)

In [177]:
make_table(control_prompts, control_answers, control_ans_prompts, "Control Names")

                                  Control Names: Ave logit diff = 3.552                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                        ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When John and Mary went to the shops, John gave the bag to    │  Mary   │  John     │ 3.337            │
│ When John and Mary went to the shops, Mary gave the bag to    │  John   │  Mary     │ 3.202            │
│ When Tom and James went to the park, James gave the ball to   │  Tom    │  James    │ 2.709            │
│ When Tom and James went to the park, Tom gave the ball to     │  James  │  Tom      │ 3.797            │
│ When Dan and Sid went to the shops, Sid gave an apple to      │  Dan    │  Sid      │ 1.720            │
│ When Dan and Sid went to the shops, Dan gave an apple to      │  Sid    │  Dan      │ 5.281            │
│ After Martin and Amy went to the park, Amy gave a drink to    │  Martin │  Amy      │ 2.601            │
│ After Martin and Amy went to the park, Martin gave a drink to │  Amy    │  Martin   │ 5.767            │
└───────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

In [183]:
[(model.to_str_tokens(ans_toks_n[k],ans_n[k]), ans_n[k]) for k in range(len(ans_toks_n))]

[([' flower', ' church'], (' church', ' flower')),
 ([' church', ' flower'], (' flower', ' church')),
 ([' goose', ' chair'], (' chair', ' goose')),
 ([' chair', ' goose'], (' goose', ' chair')),
 ([' goose', ' flower'], (' flower', ' goose')),
 ([' flower', ' goose'], (' goose', ' flower')),
 ([' bag', ' statue'], (' statue', ' bag')),
 ([' statue', ' bag'], (' bag', ' statue')),
 ([' google', ' flower'], (' flower', ' google')),
 ([' flower', ' google'], (' google', ' flower')),
 ([' flower', ' goose'], (' goose', ' flower')),
 ([' goose', ' flower'], (' flower', ' goose')),
 ([' chair', ' statue'], (' statue', ' chair')),
 ([' statue', ' chair'], (' chair', ' statue')),
 ([' google', ' flower'], (' flower', ' google')),
 ([' flower', ' google'], (' google', ' flower')),
 ([' statue', ' lamp'], (' lamp', ' statue')),
 ([' lamp', ' statue'], (' statue', ' lamp')),
 ([' lamp', ' church'], (' church', ' lamp')),
 ([' church', ' lamp'], (' lamp', ' church'))]

In [184]:
make_table(prompts_n, ans_n, ans_toks_n, "nouns")
make_table(prompts_pn, ans_pn, ans_toks_pn, "proper nouns")
make_table(prompts_rel, ans_rel, ans_toks_rel, "Religious Names")
make_table(prompts_pl, ans_pl, ans_toks_pl, "Place Names")

                                        nouns: Ave logit diff = -0.532                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                             ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  church and  flower went to the shops, flower gave the bag to │  church │  flower   │ -1.274           │
│ When  church and  flower went to the shops, church gave the bag to │  flower │  church   │ 0.807            │
│ When  chair and  goose went to the shops, goose gave the bag to    │  chair  │  goose    │ 1.643            │
│ When  chair and  goose went to the shops, chair gave the bag to    │  goose  │  chair    │ -4.941           │
│ When  flower and  goose went to the shops, goose gave the bag to   │  flower │  goose    │ 3.705            │
│ When  flower and  goose went to the shops, flower gave the bag to  │  goose  │  flower   │ -5.738           │
│ When  statue and  bag went to the shops, bag gave the bag to       │  statue │  bag      │ 2.226            │
│ When  statue and  bag went to the shops, statue gave the bag to    │  bag    │  statue   │ -4.173           │
│ When  flower and  google went to the shops, google gave the bag to │  flower │  google   │ 3.630            │
│ When  flower and  google went to the shops, flower gave the bag to │  google │  flower   │ -2.673           │
│ When  goose and  flower went to the shops, flower gave the bag to  │  goose  │  flower   │ -0.998           │
│ When  goose and  flower went to the shops, goose gave the bag to   │  flower │  goose    │ -1.724           │
│ When  statue and  chair went to the shops, chair gave the bag to   │  statue │  chair    │ -0.999           │
│ When  statue and  chair went to the shops, statue gave the bag to  │  chair  │  statue   │ -0.657           │
│ When  flower and  google went to the shops, google gave the bag to │  flower │  google   │ 3.630            │
│ When  flower and  google went to the shops, flower gave the bag to │  google │  flower   │ -2.673           │
│ When  lamp and  statue went to the shops, statue gave the bag to   │  lamp   │  statue   │ -1.419           │
│ When  lamp and  statue went to the shops, lamp gave the bag to     │  statue │  lamp     │ 1.201            │
│ When  church and  lamp went to the shops, lamp gave the bag to     │  church │  lamp     │ 0.011            │
│ When  church and  lamp went to the shops, church gave the bag to   │  lamp   │  church   │ -0.217           │
└────────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

                                     proper nouns: Ave logit diff = -2.944                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                             ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Statue and  Chair went to the shops, Chair gave the bag to   │  Statue │  Chair    │ -1.331           │
│ When  Statue and  Chair went to the shops, Statue gave the bag to  │  Chair  │  Statue   │ -1.564           │
│ When  Google and  Bag went to the shops, Bag gave the bag to       │  Google │  Bag      │ -0.151           │
│ When  Google and  Bag went to the shops, Google gave the bag to    │  Bag    │  Google   │ -0.756           │
│ When  Goose and  Statue went to the shops, Statue gave the bag to  │  Goose  │  Statue   │ -4.583           │
│ When  Goose and  Statue went to the shops, Goose gave the bag to   │  Statue │  Goose    │ -3.596           │
│ When  Flower and  Statue went to the shops, Statue gave the bag to │  Flower │  Statue   │ -3.393           │
│ When  Flower and  Statue went to the shops, Flower gave the bag to │  Statue │  Flower   │ -2.502           │
│ When  Lamp and  Statue went to the shops, Statue gave the bag to   │  Lamp   │  Statue   │ -4.809           │
│ When  Lamp and  Statue went to the shops, Lamp gave the bag to     │  Statue │  Lamp     │ -1.914           │
│ When  Bag and  Lamp went to the shops, Lamp gave the bag to        │  Bag    │  Lamp     │ -4.082           │
│ When  Bag and  Lamp went to the shops, Bag gave the bag to         │  Lamp   │  Bag      │ -4.382           │
│ When  Church and  Lamp went to the shops, Lamp gave the bag to     │  Church │  Lamp     │ -1.742           │
│ When  Church and  Lamp went to the shops, Church gave the bag to   │  Lamp   │  Church   │ -3.124           │
│ When  Goose and  Bag went to the shops, Bag gave the bag to        │  Goose  │  Bag      │ -1.604           │
│ When  Goose and  Bag went to the shops, Goose gave the bag to      │  Bag    │  Goose    │ -5.989           │
│ When  Church and  Goose went to the shops, Goose gave the bag to   │  Church │  Goose    │ -3.978           │
│ When  Church and  Goose went to the shops, Church gave the bag to  │  Goose  │  Church   │ -4.896           │
│ When  Bag and  Chair went to the shops, Chair gave the bag to      │  Bag    │  Chair    │ -5.628           │
│ When  Bag and  Chair went to the shops, Bag gave the bag to        │  Chair  │  Bag      │ 1.143            │
└────────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

                                    Religious Names: Ave logit diff = -3.378                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                              ┃ Correct  ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Jacob and  Abraham went to the shops, Abraham gave the bag to │  Jacob   │  Abraham  │ -4.699           │
│ When  Jacob and  Abraham went to the shops, Jacob gave the bag to   │  Abraham │  Jacob    │ -1.637           │
│ When  Jesus and  Jacob went to the shops, Jacob gave the bag to     │  Jesus   │  Jacob    │ -0.617           │
│ When  Jesus and  Jacob went to the shops, Jesus gave the bag to     │  Jacob   │  Jesus    │ -3.702           │
│ When  Isaac and  Jesus went to the shops, Jesus gave the bag to     │  Isaac   │  Jesus    │ -4.036           │
│ When  Isaac and  Jesus went to the shops, Isaac gave the bag to     │  Jesus   │  Isaac    │ -3.647           │
│ When  Paul and  Jacob went to the shops, Jacob gave the bag to      │  Paul    │  Jacob    │ -4.551           │
│ When  Paul and  Jacob went to the shops, Paul gave the bag to       │  Jacob   │  Paul     │ -3.103           │
│ When  Jesus and  Paul went to the shops, Paul gave the bag to       │  Jesus   │  Paul     │ 0.315            │
│ When  Jesus and  Paul went to the shops, Jesus gave the bag to      │  Paul    │  Jesus    │ -3.265           │
│ When  Noah and  Joseph went to the shops, Joseph gave the bag to    │  Noah    │  Joseph   │ -2.724           │
│ When  Noah and  Joseph went to the shops, Noah gave the bag to      │  Joseph  │  Noah     │ -5.108           │
│ When  Paul and  Joseph went to the shops, Joseph gave the bag to    │  Paul    │  Joseph   │ -2.202           │
│ When  Paul and  Joseph went to the shops, Paul gave the bag to      │  Joseph  │  Paul     │ -4.792           │
│ When  Paul and  Joseph went to the shops, Joseph gave the bag to    │  Paul    │  Joseph   │ -2.202           │
│ When  Paul and  Joseph went to the shops, Paul gave the bag to      │  Joseph  │  Paul     │ -4.792           │
│ When  Abraham and  Jesus went to the shops, Jesus gave the bag to   │  Abraham │  Jesus    │ -4.930           │
│ When  Abraham and  Jesus went to the shops, Abraham gave the bag to │  Jesus   │  Abraham  │ -3.362           │
│ When  Paul and  Isaac went to the shops, Isaac gave the bag to      │  Paul    │  Isaac    │ -5.507           │
│ When  Paul and  Isaac went to the shops, Paul gave the bag to       │  Isaac   │  Paul     │ -3.004           │
└─────────────────────────────────────────────────────────────────────┴──────────┴───────────┴──────────────────┘

                                       Place Names: Ave logit diff = -3.593                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                               ┃ Correct   ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When  Madison and  Florence went to the shops, Florence gave the bag │  Madison  │  Florence │ -2.979           │
│ to                                                                   │           │           │                  │
│ When  Madison and  Florence went to the shops, Madison gave the bag  │  Florence │  Madison  │ -6.354           │
│ to                                                                   │           │           │                  │
│ When  Madison and  Brooklyn went to the shops, Brooklyn gave the bag │  Madison  │  Brooklyn │ -2.525           │
│ to                                                                   │           │           │                  │
│ When  Madison and  Brooklyn went to the shops, Madison gave the bag  │  Brooklyn │  Madison  │ -1.968           │
│ to                                                                   │           │           │                  │
│ When  Devon and  Austin went to the shops, Austin gave the bag to    │  Devon    │  Austin   │ -2.920           │
│ When  Devon and  Austin went to the shops, Devon gave the bag to     │  Austin   │  Devon    │ -7.918           │
│ When  Austin and  Brooklyn went to the shops, Brooklyn gave the bag  │  Austin   │  Brooklyn │ -2.230           │
│ to                                                                   │           │           │                  │
│ When  Austin and  Brooklyn went to the shops, Austin gave the bag to │  Brooklyn │  Austin   │ -1.926           │
│ When  Austin and  Devon went to the shops, Devon gave the bag to     │  Austin   │  Devon    │ -2.331           │
│ When  Austin and  Devon went to the shops, Austin gave the bag to    │  Devon    │  Austin   │ -6.963           │
│ When  Austin and  Madison went to the shops, Madison gave the bag to │  Austin   │  Madison  │ -2.843           │
│ When  Austin and  Madison went to the shops, Austin gave the bag to  │  Madison  │  Austin   │ -5.580           │
│ When  Florence and  Phoenix went to the shops, Phoenix gave the bag  │  Florence │  Phoenix  │ -2.767           │
│ to                                                                   │           │           │                  │
│ When  Florence and  Phoenix went to the shops, Florence gave the bag │  Phoenix  │  Florence │ -5.405           │
│ to                                                                   │           │           │                  │
│ When  Madison and  Devon went to the shops, Devon gave the bag to    │  Madison  │  Devon    │ -3.195           │
│ When  Madison and  Devon went to the shops, Madison gave the bag to  │  Devon    │  Madison  │ -6.638           │
│ When  London and  Florence went to the shops, Florence gave the bag  │  London   │  Florence │ 1.493            │
│ to                                                                   │           │           │                  │
│ When  London and  Florence went to the shops, London gave the bag to │  Florence │  London   │ -4.285           │
│ When  Phoenix and  London went to the shops, London gave the bag to  │  Phoenix  │  London   │ -0.971           │
│ When  Phoenix and  London went to the shops, Phoenix gave the bag to │  London   │  Phoenix  │ -3.557           │
└──────────────────────────────────────────────────────────────────────┴───────────┴───────────┴──────────────────┘

Results look pretty bad across the board ... let's look at the top_k tokens to see 

In [153]:
sample_prompt = prompts_rel[5]
sample_logits = model(sample_prompt)
sample_probs = t.log_softmax(sample_logits[0, -1], dim = -1)
vals, ids = t.topk(sample_probs,5)
print(vals, sample_probs)
print([model.tokenizer.decode(ids[j]) for j in range(len(ids))])
print(sample_probs[ids[0]])



tensor([-1.2884, -1.5528, -2.2378, -2.4683, -3.5000]) tensor([-12.5240, -12.0724, -14.4456,  ..., -16.4893, -17.4218, -12.1294])
[' Abraham', ' ', ' them', ' the', ' Paul']
tensor(-1.2884)
